# Image Endpoint

Creates HTTP endpoint to accept images and call a model

In [ ]:
import os
"""
os.environ['create_image']='True'
os.environ['repository']='romeokienzler'
os.environ['version']='0.14'
"""
#os.environ['install_requirements']='True'

In [ ]:
if bool(os.environ.get('create_image',False)):
    docker_file="""
    FROM ultralytics/yolov5:latest
    RUN pip install ipython nbformat flask
    ADD image-endpoint.ipynb /
    ENTRYPOINT ["ipython","/image-endpoint.ipynb","> /tmp/component.log","2> /tmp/component.err"]
    """
    with open("Dockerfile", "w") as text_file:
        text_file.write(docker_file)

    !docker build -t claimed-predict-image-endpoint:`echo $version` .
    !docker tag claimed-predict-image-endpoint:`echo $version` `echo $repository`/claimed-predict-image-endpoint:`echo $version`
    !docker push `echo $repository`/claimed-predict-image-endpoint:`echo $version`
elif bool(os.environ.get('install_requirements',False)):
    !pip install flask

In [ ]:
from flask import Flask
from flask import request
from flask import send_file

In [ ]:
index_html = """
<html>
    <head></head>
    <body>
        <form action="process-image" method="post" enctype="multipart/form-data">
            <input type="file" name="file" id="file">
            <input type="submit" value="Upload Image" name="submit">
        </form>
    </body>
</html>
"""

index_html_labels = """
<html>
    <head></head>
    <body>
        <form action="process-labels" method="post" enctype="multipart/form-data">
            <input type="file" name="file" id="file">
            <input type="submit" value="Upload Image" name="submit">
        </form>
    </body>
</html>
"""

In [ ]:
class_names=  ['person', 'bicycle', 'car', 'motorcycle', 'airplane', 'bus', 'train', 'truck', 'boat', 'traffic light',
        'fire hydrant', 'stop sign', 'parking meter', 'bench', 'bird', 'cat', 'dog', 'horse', 'sheep', 'cow',
        'elephant', 'bear', 'zebra', 'giraffe', 'backpack', 'umbrella', 'handbag', 'tie', 'suitcase', 'frisbee',
        'skis', 'snowboard', 'sports ball', 'kite', 'baseball bat', 'baseball glove', 'skateboard', 'surfboard',
        'tennis racket', 'bottle', 'wine glass', 'cup', 'fork', 'knife', 'spoon', 'bowl', 'banana', 'apple',
        'sandwich', 'orange', 'broccoli', 'carrot', 'hot dog', 'pizza', 'donut', 'cake', 'chair', 'couch',
        'potted plant', 'bed', 'dining table', 'toilet', 'tv', 'laptop', 'mouse', 'remote', 'keyboard', 'cell phone',
        'microwave', 'oven', 'toaster', 'sink', 'refrigerator', 'book', 'clock', 'vase', 'scissors', 'teddy bear',
        'hair drier', 'toothbrush']

In [ ]:
app = Flask(__name__)

@app.route('/')
def index():
    return index_html

@app.route('/labels')
def labels():
    return index_html_labels


@app.route('/process-image', methods=['POST'])
def upload_file():
    file = request.files['file']
    file.save('/tmp/image.png')
    !rm -Rf /tmp/exp
    !python /usr/src/app/detect.py --save-txt --project /tmp/ --source /tmp/image.png
    return send_file('/tmp/exp/image.png', mimetype='image/png')


@app.route('/process-labels', methods=['POST'])
def upload_file_labels():
    file = request.files['file']
    file.save('/tmp/image.png')
    !rm -Rf /tmp/exp
    !python /usr/src/app/detect.py --save-txt --project /tmp/ --source /tmp/image.png
    return_string = ""
    with open("/tmp/exp/labels/image.txt", "r") as file:
        for line in file:
            return_string = return_string + class_names[int(line[0])] + " " + line + '<br>'
    return return_string


app.run(host='0.0.0.0', port=8080)